# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм со столбцами: <дата> - <заголовок> - <ссылка>.

In [3]:
import requests
import time
import pandas as pd
from bs4 import BeautifulSoup
from datetime import datetime

In [4]:
# для простоты исходим из того, что искомые слова могут быть в заголовке, тексте, хабах...
KEYWORDS = ['python', 'парсинг']
all_news = pd.DataFrame()

# на хабре, из того, что увидел, 50 страниц на все новости (можно сделать бесконечный цикл, но мне они не нравятся)
for page in range(50):
    time.sleep(0.2)
    soup = BeautifulSoup(requests.get(f'https://habr.com/ru/all/page{page+1}/').text, 'html.parser')
    for post in soup.find_all('article', class_='post post_preview'):
        if any([kwd in str(post).lower() for kwd in KEYWORDS]):
            all_news = (
                pd.concat([all_news,
                pd.DataFrame([
                {'page':str(page+1),
                'date': post.find('span', class_='post__time').text,
                'title': post.find('a', class_='post__title_link').text,
                'url': post.find('a', class_='post__title_link').attrs.get('href')}])],
                ignore_index=True)
                    )
all_news

,page,date,title,url
0,1,вчера в 20:22,Киоск Raspberry Pi для графического интерфейса...,https://habr.com/ru/post/516542/
1,2,вчера в 16:16,Разбор особенностей официального Docker-образа...,https://habr.com/ru/company/ruvds/blog/516310/
2,2,вчера в 16:14,"Полноценная игра, сделанная мною в обычной win...",https://habr.com/ru/post/516548/
3,2,вчера в 14:53,3D ML. Часть 3: датасеты и фреймворки в 3D ML,https://habr.com/ru/company/itmai/blog/516404/
4,3,вчера в 13:44,Анализ сетей с использованием графов,https://habr.com/ru/post/516514/
...,...,...,...,...
65,49,5 августа 2020 в 16:48,8 продвинутых возможностей модуля logging в Py...,https://habr.com/ru/post/513966/
66,50,5 августа 2020 в 14:00,"Нет времени объяснять, сделай автопилот",https://habr.com/ru/company/ods/blog/513604/
67,50,5 августа 2020 в 13:47,Дешифровка текста методом частотного анализа,https://habr.com/ru/post/513926/
68,50,5 августа 2020 в 13:10,"Canary деплой с Jenkins-X, Istio и Flagger",https://habr.com/ru/company/nixys/blog/513918/


### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.

Итоговый датафрейм формировать со столбцами: <дата> - <заголовок> - <ссылка> - <текст статьи>

In [5]:
# окай. надо просто добавить переход по ссылкам
KEYWORDS = ['python', 'парсинг']
all_news = pd.DataFrame()

# ограничимся 2мя страницами для примера, но можно собрать и со всех
# можно через функцию, но проще 2 строчки добавить к имеющемуся скрипту
for page in range(2):
    soup = BeautifulSoup(requests.get(f'https://habr.com/ru/all/page{page+1}/').text, 'html.parser')
    for post in soup.find_all('article', class_='post post_preview'):
        time.sleep(0.2)        
        soup_full = BeautifulSoup(requests.get(post.find('a', class_='post__title_link').attrs.get('href')).text, 'html.parser')
        post_full = soup_full.find('article', class_='post post_full')
        if any([kwd in str(post_full).lower() for kwd in KEYWORDS]):
            all_news = (
                pd.concat([all_news,
                pd.DataFrame([
                {'page':str(page+1),
                'date': post.find('span', class_='post__time').text,
                'title': post.find('a', class_='post__title_link').text,
                'url': post.find('a', class_='post__title_link').attrs.get('href'),
                'body': post_full.find('div', class_='post__text').text,}])],
                ignore_index=True)
                    )
all_news

,page,date,title,url,body
0,1,вчера в 20:22,Киоск Raspberry Pi для графического интерфейса...,https://habr.com/ru/post/516542/,"Привет, Хабр!\n\r\nХочется поделиться опытом н..."
1,1,вчера в 18:44,7 интересных хаков с Black Hat / DEF CON 2020,https://habr.com/ru/company/it-grad/blog/516570/,Под катом приглядимся повнимательнее к некотор...
2,1,вчера в 17:41,Обратный перевод для Нейронного машинного пере...,https://habr.com/ru/post/491794/,Привет. Некоторое время назад я рассказывал пр...
3,2,вчера в 16:16,Разбор особенностей официального Docker-образа...,https://habr.com/ru/company/ruvds/blog/516310/,Официальный Docker-образ Python весьма популяр...
4,2,вчера в 16:14,"Полноценная игра, сделанная мною в обычной win...",https://habr.com/ru/post/516548/,"Привет! \r\nСегодня я опишу в подробностях, ка..."
5,2,вчера в 15:10,Azure IoT Edge и SQL Edge: перенос облачных на...,https://habr.com/ru/company/quarta/blog/516432/,"Azure IoT Edge — это дополнение к IoT Hub, кот..."
6,2,вчера в 14:53,3D ML. Часть 3: датасеты и фреймворки в 3D ML,https://habr.com/ru/company/itmai/blog/516404/,\nРаботая в конкретной предметной области в ра...
7,2,вчера в 14:45,Телепортация тонн данных в PostgreSQL,https://habr.com/ru/company/tensor/blog/516384/,Сегодня я поделюсь некоторыми полезными архите...


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: <почта> - <дата утечки> - <источник утечки> - <описание утечки>.

In [7]:
EMAIL = ['test@ya.ru', 'test@mail.ru']
df_res = pd.DataFrame()

URL = 'https://digibody.avast.com/v1/web/leaks'

for eml in EMAIL:
    time.sleep(0.2)
    data = {'email': eml}
    req = requests.post(URL, json=data)    
    
    df = pd.DataFrame(req.json()['value'])
    df['date of leak'], df['title'], df['description'] = (
        zip(*df.apply(lambda x:
        (datetime.fromtimestamp(x.leak_date/1000).strftime('%Y-%m-%d'),
        x['leak_info']['title'],
        x['leak_info']['description']), axis=1))
    )
    # в результатах высвечиваются какие-то левые e-mail. на всякий, я их убрал. можно это не делать
    df = df[df.username==eml][['username', 'date of leak', 'domain', 'title', 'description']]
    df_res = pd.concat([df_res, df], ignore_index=True)

df_res.columns = ['почта', 'дата утечки', 'источник утечки', 'описание утечки', 'описание утечки полное']
df_res

,почта,дата утечки,источник утечки,описание утечки,описание утечки полное
0,test@ya.ru,2018-08-24,,Jadid List from Pemiblanc.com,This combolist was compiled from a variety of ...
1,test@ya.ru,2019-02-06,,Collection #4 Combo List,"On January 7, 2019, an online user named Sanix..."
2,test@ya.ru,2017-01-16,aristos.pw,Aristos,"In September 2016, Aristos.pw was breached. Th..."
3,test@ya.ru,2020-05-21,bitrewards.com,BitRewards,"In April 2020, the e-commerce platform BitRewa..."
4,test@ya.ru,2016-10-24,dropbox.com,Dropbox,Cloud storage company Dropbox suffered a major...
5,test@ya.ru,2018-12-13,,Sensitive Source,This source has been marked as sensitive due t...
6,test@ya.ru,2019-02-06,,Collection #5 Combo List,"On January 7, 2019, an online user named Sanix..."
7,test@ya.ru,2017-04-04,evony.com,Evony,"In July 2016, gaming site Evony's user databas..."
8,test@ya.ru,2020-01-09,,Sensitive Source,This source has been marked as sensitive due t...
9,test@ya.ru,2019-02-06,,2019 Antipublic Combo List,"On January 7, 2019, an online user named Sanix..."


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)```
GROUP = 'netology'
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ
```

В итоге должен формироваться датафрейм со столбцами: <дата поста> - <текст поста>.

In [1]:
# решил таки заморочиться с получением токена
NEWSFEED_REQUEST = 'https://api.vk.com/method/wall.get?'
TOKEN = 'был тут'
VERSION = '5.122'
SLEEP = 0.33

In [21]:
params = {
    'access_token': TOKEN,
    'domain': 'netology',
    'v': VERSION,
    'count': 50
}
res = requests.get(NEWSFEED_REQUEST, params)
df = pd.DataFrame(res.json()['response']['items'])
df['dt'] = df.apply(lambda x: datetime.fromtimestamp(x.date).strftime('%Y-%m-%d'), axis=1)
df = df[['dt', 'text']]
df.columns = ['дата поста', 'текст поста']
df

,дата поста,текст поста
0,2020-08-22,А ю рэди? 🕺 На следующей неделе учимся выступа...
1,2020-08-26,"Казалось бы, общение — это просто. Кто-то гово..."
2,2020-08-25,"Любимая работа, амбициозные проекты, полное по..."
3,2020-08-25,"Буквы разные писать, тонким пёрышком в тетрадь..."
4,2020-08-24,SMM — это не только про креатив. Здесь важен р...
5,2020-08-23,Подкасты — отличный способ узнавать что-то нов...
6,2020-08-23,Вместе с командой приложения для проектировани...
7,2020-08-22,*партнерский пост* \n \n17-18 сентября пройдёт...
8,2020-08-22,"Ладони потеют, колени подкашиваются, голос дро..."
9,2020-08-21,Итоги карьерного квеста Нетологии🎉 \n \nМы хот...
